MODIFICACION  DE DATOS

In [2]:
import pandas as pd
import ast
import datetime
import numpy as np

In [3]:
df = pd.read_csv(r'proyecto DFs\movies_dataset.csv')
df2= pd.read_csv(r'proyecto DFs\credits1.csv')
df3= pd.read_csv(r'proyecto DFs\credits2.csv')

df2 = pd.concat([df2, df3], ignore_index=True)

C:\Users\alan\AppData\Local\Temp\ipykernel_23856\2503794693.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'proyecto DFs\movies_dataset.csv')


In [4]:
df.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [20]:
df['id'] = pd.to_numeric(df['id'], errors='coerce').fillna(0).astype(int)

In [21]:
df = df.merge(df2, on='id')

PUNTO 1 Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [22]:
def extraer_belongs_to_collection(cell):
    if pd.isnull(cell):
        return ""
    else:
        collection = ast.literal_eval(cell)
        return collection['name']

df['belongs_to_collection'] = df['belongs_to_collection'].apply(extraer_belongs_to_collection)

In [23]:
def extraer_genres(cell):
    if pd.isnull(cell):
        return []
    else:
        genres = ast.literal_eval(cell)
        names = [genre['name'] for genre in genres]
        return names

df['genres'] = df['genres'].apply(extraer_genres)

In [24]:
def extraer_production_companies(cell):
    if pd.isnull(cell):
        return []
    else:
        companies = ast.literal_eval(cell)
        names = [companie['name'] for companie in companies]
        return names

df['production_companies'] = df['production_companies'].apply(extraer_production_companies)

In [25]:
def extraer_production_countries(cell):
    if pd.isnull(cell):
        return []
    else:
        countries = ast.literal_eval(cell)
        names = [country['name'] for country in countries]
        return names

df['production_countries'] = df['production_countries'].apply(extraer_production_countries)

In [26]:
def extraer_spoken_languages(cell):
    if pd.isnull(cell):
        return []
    else:
        languages = ast.literal_eval(cell)
        names = [language['name'] for language in languages]
        return names

df['spoken_languages'] = df['spoken_languages'].apply(extraer_spoken_languages)


In [27]:
def extraer_cast(cell):
    if pd.isnull(cell):
        return []
    else:
        cast = ast.literal_eval(cell)
        names = [person['name'] for person in cast]
        return names

df['cast'] = df['cast'].apply(extraer_cast)

In [28]:
def extraer_crew(cell):
    if pd.isnull(cell):
        return []
    else:
        crew = ast.literal_eval(cell)
        names = [person['name'] for person in crew]
        return names

df['crew'] = df['crew'].apply(extraer_crew)

In [29]:
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J..."
1,False,,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho..."
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev..."


PUNTO 2
Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0

In [30]:
df['revenue'] = df['revenue'].fillna(0)
df['budget'] = df['budget'].fillna(0)

PUNTO 3 Los valores nulos del campo release date deben eliminarse.

In [31]:
df = df.dropna(subset=['release_date'])

PUNTO 4 De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [32]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce', format='%Y-%m-%d')

In [33]:
df['release_year'] = df['release_date'].dt.year

In [34]:
df['release_year'] = df['release_year'].astype('Int64')

PUNTO 5 Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [35]:
df['budget'] = pd.to_numeric(df['budget'], errors='coerce').fillna(0).astype(float)

In [36]:
df['return'] = np.divide(df['revenue'], df['budget']).fillna(0)

In [37]:
df['return'] = df['return'].replace([np.inf, -np.inf], 0)

PUNTO 6 Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage.

In [38]:
df = df.drop(['video','imdb_id','adult','original_title','poster_path', 'homepage'], axis=True)

In [39]:
df.head(5)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,[English],Released,NaN,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...",1995,12.451801
1,,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho...",1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev...",1995,0.000000
3,,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...",1995,5.090760
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N...",1995,0.000000


In [40]:
df.to_csv(r'C:\Users\alan\tareas jupyter notebook\Proyecto 1\df_modificado.csv', index=False)